<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/Attention_attribution_cosine_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook adapts the Captum tutorial for question answering and refactors it into the longformer sequence classification task. Specifically, this notebook focuses on using the model's embeddings to get word attributions for the examples of your choice, or the entire dataset if needed.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import dependencies

In [2]:
pip install transformers --quiet

In [3]:
pip install captum --quiet

In [4]:
pip install datasets --quiet

In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [6]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch
import pandas as pd

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Import model

In [8]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'danielhou13/longformer-finetuned_papers_v2'
#model_path = 'danielhou13/longformer-finetuned-new-cogs402'

# load model
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Create functions that give us the input ids and the position ids for the text we want to examine

In [9]:
def predict(inputs, position_ids=None, attention_mask=None):
    output = model(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    return output.logits

In [10]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [11]:
max_length = 2046
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)

    #taken from the longformer implementation
    mask = input_ids.ne(ref_token_id).int()
    incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask
    position_ids = incremental_indices.long().squeeze() + ref_token_id

    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

Import dataset and take a few examples from it for testing purposes

Here we import the papers dataset

In [12]:
from datasets import load_dataset
import numpy as np
cogs402_ds = load_dataset("danielhou13/cogs402dataset")["test"]

Using custom data configuration danielhou13--cogs402dataset-144b958ac1a53abb
Reusing dataset parquet (/root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402dataset-144b958ac1a53abb/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/2 [00:00<?, ?it/s]

Here we import the news dataset

In [13]:
# cogs402_ds2 = load_dataset('hyperpartisan_news_detection', 'bypublisher')['validation']
# val_size = 5000
# val_indices = np.random.randint(0, len(cogs402_ds2), val_size)
# val_ds = cogs402_ds2.select(val_indices)
# labels2 = map(int, val_ds['hyperpartisan'])
# labels2 = list(labels2)
# val_ds = val_ds.add_column("labels", labels2)

In [14]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None, attention_mask=None):
    preds = predict(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask
                   )
    return torch.softmax(preds, dim = 1)

Perform Layer Integrated Gradients using the longformer's embeddings

In [15]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    return attributions

In [16]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

This function will let us get the example and the baseline inputs in order to perform integrated gradients, and add the attributions to our visualization tool. Additionally, we will add the attributions and tokens for each example into an array so we can use them when we want to further example the attributions scores for each example

In [17]:
all_attributions = {}
all_tokens = {}

In [ ]:
# all_attributions = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/example_attrib_dict.pt')

In [530]:
example = 148
text = cogs402_ds['text'][example]
label = cogs402_ds['labels'][example]

input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens_curr = tokenizer.convert_ids_to_tokens(indices)

all_tokens[str(example)] = all_tokens_curr

In [531]:
# attributions, delta = lig.attribute(inputs=input_ids,
#                                   baselines=ref_input_ids,
#                                   return_convergence_delta=True,
#                                   additional_forward_args=(position_ids, attention_mask),
#                                   target=1,
#                                   n_steps=1000,
#                                   internal_batch_size = 2)

# attributions_sum = summarize_attributions(attributions)

# all_attributions[str(example)] = attributions_sum.detach().cpu().numpy()

In [532]:
# torch.save(all_attributions, '/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/example_attrib_dict.pt')

In [533]:
all_attributions = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/example_attrib_dict.pt')

We then get the attentions and global attentions so we can compare with the attributions

In [534]:
output = model(input_ids.cuda(), attention_mask=attention_mask.cuda(), labels=torch.tensor(label).cuda(), output_attentions = True)
batch_attn = output[-2]
output_attentions = torch.stack(batch_attn).cpu()
global_attention = output[-1]
output_global_attentions = torch.stack(global_attention).cpu()
print("output_attention.shape", output_attentions.shape)
print("gl_output_attention.shape", output_global_attentions.shape)

output_attention.shape torch.Size([12, 1, 12, 2048, 514])
gl_output_attention.shape torch.Size([12, 1, 12, 2048, 1])


Since the longformer has a unique attention matrix shape, we convert it into the required sequence length x sequence length matrix

In [535]:
def create_head_matrix(output_attentions, global_attentions):
    new_attention_matrix = torch.zeros((output_attentions.shape[0], 
                                      output_attentions.shape[0]))
    for i in range(output_attentions.shape[0]):
        test_non_zeroes = torch.nonzero(output_attentions[i]).squeeze()
        test2 = output_attentions[i][test_non_zeroes[1:]]
        new_attention_matrix_indices = test_non_zeroes[1:]-257 + i
        new_attention_matrix[i][new_attention_matrix_indices] = test2
        new_attention_matrix[i][0] = output_attentions[i][0]
        new_attention_matrix[0] = global_attentions.squeeze()[:output_attentions.shape[0]]
    return new_attention_matrix


def attentions_all_heads(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = create_head_matrix(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_batches(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = attentions_all_heads(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_layers(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = all_batches(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

In [536]:
converted_mat = all_layers(output_attentions, output_global_attentions).detach().cpu().numpy()
print(converted_mat.shape)

(12, 1, 12, 2048, 2048)


We scale the attention matrix by head importance

In [537]:
head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/papers/pretrained/head_importance.pt")
# head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/news/head_importance.pt")

In [538]:
def scale_by_importance(attention_matrix, head_importance):
  new_matrix = np.zeros_like(attention_matrix)
  for i in range(attention_matrix.shape[0]):
    head_importance_layer = head_importance[i]
    for j in range(attention_matrix.shape[1]):
      new_matrix[i,j] = attention_matrix[i,j] * np.expand_dims(head_importance_layer, axis=(1,2))
  return new_matrix

In [539]:
converted_mat_importance = scale_by_importance(converted_mat, head_importance)

We get the attentions for each token. The shape of the attention matrix is layer x batch x head x seq_len x seq_len.

In [540]:
attention_matrix_importance = converted_mat_importance.sum(axis=3)
print(attention_matrix_importance.shape)

(12, 1, 12, 2048)


Sum the attentions for the last layer and over all layers

In [541]:
attention_final_layer = attention_matrix_importance[11].squeeze().sum(axis=0)
attention_all_layer = attention_matrix_importance.squeeze().sum(axis=1)
attention_all_layer = attention_all_layer.sum(axis=0)
print(attention_all_layer.shape)

(2048,)


Grab the attributions we stored

In [542]:
exam_attrib = all_attributions[str(example)]

Since we have the attributions and the attentions, we want to see how the largest attributions (in terms of magnitude) compares to the highest attentions.

The attributions and the attentions have different ranges. The attributions could range from -1 to 1 whereas the attentions range from 0 to 1. However, negative attributions would not necessarily mean that they have the lowest attention, rather they might have really high attention as they are more likely to help the model predict the negative class, and might be something the attentions picked up as a feature.

In [543]:
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [544]:
attention_final_layer2 = normalize(attention_final_layer)
attention_all_layer2 = normalize(attention_all_layer)

In [545]:
exam_attrib2 = np.abs(exam_attrib)
exam_attrib2 = normalize(exam_attrib2)

In [546]:
print(exam_attrib2)

[0.         0.00834903 0.01240896 ... 0.00058866 0.00076564 0.        ]


Cosine similarity using the raw attributions and attentions

In [547]:
from numpy.linalg import norm
cosine_raw = np.dot(exam_attrib, attention_final_layer) / (norm(exam_attrib)*norm(attention_final_layer))
print("Layer 12 Cosine Similarity raw attrib:\n", cosine_raw)
cosine_all_raw = np.dot(exam_attrib, attention_all_layer) / (norm(exam_attrib)*norm(attention_all_layer))
print("Layer 12 Cosine Similarity raw attrib:\n", cosine_all_raw)

Layer 12 Cosine Similarity raw attrib:
 -0.018103517675378412
Layer 12 Cosine Similarity raw attrib:
 0.01632904521786691


Calculate cosine simularity using normalized attentions and attributions

In [548]:
cosine = np.dot(exam_attrib2, attention_final_layer2) / (norm(exam_attrib2)*norm(attention_final_layer2))
print("Layer 12 Cosine Similarity:\n", cosine)
cosine2 = np.dot(exam_attrib2, attention_all_layer2) / (norm(exam_attrib2)*norm(attention_all_layer2))
print("All layer Cosine Similarity:\n", cosine2)

Layer 12 Cosine Similarity:
 0.1473134137172987
All layer Cosine Similarity:
 0.14074884787451916


Cosine similarity while setting all the attention and attribution values below the median to 0

In [549]:
exam_attrib3 = np.abs(exam_attrib)
exam_attrib3 = normalize(exam_attrib3)
median_exam = np.percentile(exam_attrib3, 50)
exam_attrib3[exam_attrib3 < median_exam] = 0

In [550]:
attention_final_layer3 = np.copy(attention_final_layer)
attention_final_layer3 = normalize(attention_final_layer3)
median_12 = np.percentile(attention_final_layer3, 50)
attention_final_layer3[attention_final_layer3 < median_12] = 0

attention_all_layer3 = np.copy(attention_all_layer) 
attention_all_layer3 = normalize(attention_all_layer3)
median_all = np.percentile(attention_all_layer3, 50)
attention_all_layer3[attention_all_layer3 < median_all] = 0

In [551]:
cosine_med = np.dot(exam_attrib3, attention_final_layer3) / (norm(exam_attrib3)*norm(attention_final_layer3))
print("Layer 12 Cosine Similarity med:\n", cosine_med)
cosine_med2 = np.dot(exam_attrib3, attention_all_layer3) / (norm(exam_attrib3)*norm(attention_all_layer3))
print("All layer Cosine Similarity med:\n", cosine_med2)

Layer 12 Cosine Similarity med:
 0.14278686722191247
All layer Cosine Similarity med:
 0.13420874808598598


Cosine similarity while setting all the attention and attribution values below the mean to 0

In [552]:
exam_attrib4 = np.abs(exam_attrib)
exam_attrib4 = normalize(exam_attrib4)
mean_exam = np.mean(exam_attrib4)
exam_attrib4[exam_attrib4 < mean_exam] = 0

In [553]:
attention_final_layer4 = np.copy(attention_final_layer)
attention_final_layer4 = normalize(attention_final_layer4)
mean_12 = np.mean(attention_final_layer4)
attention_final_layer4[attention_final_layer4 < mean_12] = 0

attention_all_layer4 = np.copy(attention_all_layer) 
attention_all_layer4 = normalize(attention_all_layer4)
mean_all = np.mean(attention_all_layer4)
attention_all_layer4[attention_all_layer4 < mean_all] = 0

In [554]:
cosine_mean = np.dot(exam_attrib4, attention_final_layer4) / (norm(exam_attrib4)*norm(attention_final_layer4))
print("Layer 12 Cosine Similarity mean:\n", cosine_mean)
cosine_mean2 = np.dot(exam_attrib4, attention_all_layer4) / (norm(exam_attrib4)*norm(attention_all_layer4))
print("All layer Cosine Similarity mean:\n", cosine_mean2)

Layer 12 Cosine Similarity mean:
 0.12255442461316103
All layer Cosine Similarity mean:
 0.12389528720314427


The cosine similarity using only the last layer of attentions

In [555]:
d = {'example': [example], 'similarity normalized': [cosine], 'similarity raw': [cosine_raw], 'sim_norm w/ median threshold':[cosine_med], 'sim_norm w/ mean threshold':[cosine_mean]}
df = pd.DataFrame(data=d)
df

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold
0,148,0.147313,-0.018104,0.142787,0.122554


The cosine similarity using all layers

In [556]:
d2 = {'example': [example], 'similarity normalized': [cosine2], 'similarity raw': [cosine_all_raw], 'sim_norm w/ median threshold':[cosine_med2], 'sim_norm w/ mean threshold':[cosine_mean2]}
df2 = pd.DataFrame(data=d2)
df2

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold
0,148,0.140749,0.016329,0.134209,0.123895


In [557]:
df_layer12 = pd.read_csv("/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/cos_sim_layer12.csv")
df_all = pd.read_csv("/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/cos_sim_all.csv")

In [558]:
df_layer12

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold
0,148,0.147313,-0.018104,0.142787,0.122554
1,605,0.380512,-0.114940,0.370985,0.351892
2,891,0.166272,0.015165,0.164239,0.147608
3,976,0.138272,0.074832,0.134038,0.116042


In [559]:
df_all

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold
0,148,0.140749,0.016329,0.134209,0.123895
1,605,0.239283,-0.185178,0.224306,0.211469
2,891,0.103062,-0.014182,0.099462,0.088445
3,976,0.114463,0.095076,0.108553,0.096792


Append the new row into the dataframe.

Comment out if revisiting a dataframe.

In [560]:
# df_layer12 = pd.concat([df, df_layer12], axis=0)
# df_all = pd.concat([df2, df_all], axis=0)

In [561]:
df_layer12

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold
0,148,0.147313,-0.018104,0.142787,0.122554
1,605,0.380512,-0.114940,0.370985,0.351892
2,891,0.166272,0.015165,0.164239,0.147608
3,976,0.138272,0.074832,0.134038,0.116042


In [562]:
df_all

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold
0,148,0.140749,0.016329,0.134209,0.123895
1,605,0.239283,-0.185178,0.224306,0.211469
2,891,0.103062,-0.014182,0.099462,0.088445
3,976,0.114463,0.095076,0.108553,0.096792


In [563]:
df_layer12 = df_layer12.sort_values(by=['example'])
df_all = df_all.sort_values(by=['example'])

Save the dataframe

In [564]:
# df_layer12.to_csv("/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/cos_sim_layer12.csv", index=False)
# df_all.to_csv("/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/cos_sim_all.csv", index=False)

We know from the cosine similarities that it does not seem like the attribtions and the attentions are very similar; however, we can find out if there are similarities in the tokens in the highest percentiles.


In [565]:
attention_final_layer5 = np.copy(attention_final_layer)
attention_final_layer5 = normalize(attention_final_layer5)

attention_all_layer5 = np.copy(attention_all_layer) 
attention_all_layer5 = normalize(attention_all_layer5)

exam_attrib5 = np.abs(exam_attrib)
exam_attrib5 = normalize(exam_attrib5)
print(exam_attrib5)

[0.         0.00834903 0.01240896 ... 0.00058866 0.00076564 0.        ]


In [566]:
top_final = np.percentile(attention_final_layer5, 95)
top_all = np.percentile(attention_final_layer5, 95)
top_attrib = np.percentile(exam_attrib5, 95)
print(top_attrib)

0.04110371800201079


In [567]:
attention_final_layer5[attention_final_layer5<top_final] = 0
attention_all_layer5[attention_all_layer5<top_all] = 0
exam_attrib5[exam_attrib5<top_attrib] = 0

In [568]:
print(exam_attrib5)

[0. 0. 0. ... 0. 0. 0.]


In [569]:
cosine_thresh = np.dot(exam_attrib5, attention_final_layer5) / (norm(exam_attrib5)*norm(attention_final_layer5))
print("Layer 12 Cosine Similarity 95th:\n", cosine_thresh)
cosine_thresh2 = np.dot(exam_attrib5, attention_all_layer5) / (norm(exam_attrib5)*norm(attention_all_layer5))
print("All layer Cosine Similarity 95th:\n", cosine_thresh2)

Layer 12 Cosine Similarity 95th:
 0.0737251888541358
All layer Cosine Similarity 95th:
 0.0


In [570]:
attention_final_layer_top = np.flatnonzero(attention_final_layer5)
attention_final_layer_top = set(attention_final_layer_top)

attention_all_layer_top = np.flatnonzero(attention_all_layer5)
attention_all_layer_top = set(attention_all_layer_top)

exam_attrib_top = np.flatnonzero(exam_attrib5)
exam_attrib_top = set(exam_attrib_top)
print(exam_attrib_top)

{1032, 1033, 1034, 1036, 535, 1048, 1049, 1050, 1079, 1593, 1085, 1090, 1093, 70, 1094, 74, 1615, 83, 1622, 1114, 97, 1641, 1643, 108, 1132, 1646, 113, 114, 115, 116, 1650, 1655, 1657, 123, 1659, 1660, 1150, 638, 127, 131, 1157, 1671, 649, 139, 140, 142, 143, 144, 1690, 1185, 162, 1187, 1188, 169, 170, 171, 172, 1194, 177, 179, 182, 183, 184, 185, 1719, 187, 1724, 1725, 1726, 193, 194, 1742, 1744, 1745, 1252, 1270, 759, 767, 1792, 260, 1808, 1835, 1848, 1860, 1863, 328, 1873, 866, 928, 1970, 967, 971, 972, 973, 974, 976, 978, 979, 1494, 987, 997, 1015, 1023}


Grab the tokens stored in the all tokens dictionary so we can know which tokens we are working with as we currently only have the indices.

In [571]:
exam_tokens = all_tokens[str(example)]

Find out which tokens have the highest attentions but not the highest attributions

In [572]:
diff = sorted(list(attention_final_layer_top - exam_attrib_top))
print(len(diff))
diff_tokens = [exam_tokens[idx] for idx in diff]
d_diff = {"token": diff_tokens, "position":diff, "attention_norm":attention_final_layer5[diff], "attribution_norm":exam_attrib5[diff]}
df_diff = pd.DataFrame(d_diff)
df_diff

83


,token,position,attention_norm,attribution_norm
0,<s>,0,0.290211,0.0
1,.,93,0.502393,0.0
2,.,145,0.518136,0.0
3,.,163,0.534227,0.0
4,.,231,0.571874,0.0
...,...,...,...,...
78,Ġlanguages,1904,0.331379,0.0
79,.,1940,0.582864,0.0
80,.,1972,0.535263,0.0
81,.,2022,0.459906,0.0


In [573]:
print(df_diff['token'].value_counts())

.               38
Ġthe             4
Ġprogram         2
Ġdatabase        2
<s>              1
Ġdependence      1
Ġqualitative     1
Ġbehavior        1
Ġsystems         1
ĠOpen            1
ance             1
Ġagents          1
Ġgeneration      1
Ġwork            1
Ġcontrol         1
ĠSem             1
antic            1
ĠWeb             1
Ġusers           1
Ġmotivations     1
ĠProgramming     1
flows            1
Ġprograms        1
Ġtools           1
Ġsemantics       1
Ġexecuted        1
âĢ               1
Ŀ                1
Ġresults         1
Ġsystem          1
,                1
Ġnetwork         1
Ġcomputation     1
Ġcloud           1
Ġlearn           1
ogeneous         1
Ġrun             1
Ġcomput          1
Ġsemantic        1
Ġworkflow        1
Ġlanguages       1
Name: token, dtype: int64


Find out which tokens have the highest attributions but not the highest attentions

In [574]:
diff2 = sorted(list(exam_attrib_top - attention_final_layer_top))
print(len(diff))
diff_tokens2 = [exam_tokens[idx] for idx in diff2]
d_diff2 = {"token": diff_tokens2, "position":diff2, "attention_norm": attention_final_layer5[diff2], "attribution_norm":exam_attrib5[diff2]}
df_diff2 = pd.DataFrame(d_diff2)
df_diff2

83


,token,position,attention_norm,attribution_norm
0,Ġdata,70,0.0,0.127100
1,Ġstudied,74,0.0,0.070683
2,Ġdatabases,83,0.0,0.086790
3,Ġmechanisms,97,0.0,0.079252
4,Ġinfluence,108,0.0,0.048288
...,...,...,...,...
78,Ġdata,1848,0.0,0.062267
79,Ġscientists,1860,0.0,0.046638
80,Ġscientific,1863,0.0,0.050297
81,Ġdata,1873,0.0,0.047750


In [575]:
print(df_diff2['token'].value_counts())

Ġdata            5
Ġcausal          5
Ġscientific      5
Ġprogram         4
Ġcaus            4
,                3
Ġscientists      3
Ġmechanisms      2
Ġa               2
Ġstudied         2
Ġresults         2
Ġthat            2
Ġin              2
Ġof              2
Ġmodels          2
Ġgraphs          2
ality            2
Ġand             2
Ġfor             1
Ġworkflow        1
Ġusers           1
Ġcause           1
Ġthe             1
Ġlibraries       1
Ġtheories        1
Ġprograms        1
Ġmathematical    1
ĠScientists      1
Ġuser            1
Ġover            1
ĠC               1
Ġprogrammer      1
flow             1
Ġbased           1
Ġstudy           1
Ġpolicies        1
Ġmotivating      1
Ġconcepts        1
Ġas              1
Ġtechniques      1
Ġtheory          1
Ġhas             1
Ġdescribe        1
Ġdatabases       1
Ġsemantics       1
Ġdigital         1
Ġinfluence       1
Ġhigh            1
Ġvisual          1
Ġresearch        1
Name: token, dtype: int64


Find out which tokens are part of the highest attentions and highest attributions.

In [576]:
same = sorted(list(attention_final_layer_top & exam_attrib_top))
print(len(same))
same_tokens = [exam_tokens[idx] for idx in same]
d_same = {"token": same_tokens, "position":same, "attention_norm": attention_final_layer5[same], "attribution_norm":exam_attrib5[same]}
df_same = pd.DataFrame(d_same)
df_same

20


,token,position,attention_norm,attribution_norm
0,.,116,0.511666,0.049573
1,Ġartificial,183,0.126003,0.800534
2,Ġintelligence,184,0.129568,0.425146
3,Ġprogram,535,0.740201,0.080474
4,Ġprogram,638,0.170902,0.082891
5,Ġprogram,649,0.105884,0.070410
6,Ġprogramming,972,0.129368,0.956748
7,Ġlanguages,973,0.079440,1.000000
8,Ġconcurrent,987,0.112719,0.172128
9,Ġcomputing,997,0.061421,0.091481


In [577]:
print(df_same['token'].value_counts())

Ġprogram          4
.                 2
Ġlanguages        2
Ġcausal           2
Ġartificial       1
Ġintelligence     1
Ġprogramming      1
Ġconcurrent       1
Ġcomputing        1
++                1
ĠJava             1
Ġquery            1
Ġsemantics        1
Ġpsychological    1
Name: token, dtype: int64
